In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.nn import CrossEntropyLoss

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.nn import CrossEntropyLoss

import torch
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup





class FerdowsiPoemDataset(Dataset):
    def __init__(self, poems_file, tokenizer, max_length=512):
        self.poems = self.load_poems(poems_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def load_poems(self, poems_file):
        with open(poems_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        poems = [line.strip() for line in lines]
        return poems

    def __len__(self):
        return len(self.poems)

    def __getitem__(self, idx):
        input_text = self.poems[idx]
        inputs = self.tokenizer.encode(input_text, add_special_tokens=True, truncation=True)
        return torch.tensor(inputs, dtype=torch.long)

def collate_fn(batch):
    inputs = batch

    max_length = max(len(seq) for seq in inputs)

    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)

    if inputs_padded.size(1) < max_length:
        inputs_padded = F.pad(inputs_padded, (0, max_length - inputs_padded.size(1)), value=0)

    attention_masks = (inputs_padded != 0).float()

    return inputs_padded, attention_masks



def train_model(model, train_dataloader, epochs=10, learning_rate=2e-5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()


    optimizer = AdamW(
    model.parameters(),
    lr=5e-4,
    eps=1e-8
     )
    scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=len(train_dataloader) * epochs)
    loss_fn = CrossEntropyLoss()

    for epoch in range(epochs):
       total_loss = 0
       for input_batch, attention_mask in train_dataloader:
           input_batch, attention_mask = input_batch.to(device), attention_mask.to(device)

           optimizer.zero_grad()

           outputs = model(input_batch, attention_mask=attention_mask, labels=input_batch)
           loss = outputs.loss

           loss.backward()
           optimizer.step()
           scheduler.step()

           total_loss += loss.item()

       avg_loss = total_loss / len(train_dataloader)
       print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")







In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.nn import CrossEntropyLoss
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.nn import CrossEntropyLoss

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.nn import CrossEntropyLoss

import torch
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.optim import AdamW
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

class FerdowsiPoemDataset(Dataset):
    def __init__(self, poems_file, tokenizer, max_length=512):
        self.poems = self.load_poems(poems_file)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def load_poems(self, poems_file):
        with open(poems_file, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        poems = [line.strip() for line in lines]
        return poems

    def __len__(self):
        return len(self.poems)

    def __getitem__(self, idx):
        input_text = self.poems[idx]
        tokenized = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
        return {'input_ids': tokenized['input_ids'].squeeze(), 'attention_mask': tokenized['attention_mask'].squeeze()}

def train_model(model, train_dataloader, epochs=10, learning_rate=5e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    optimizer = AdamW(
        model.parameters(),
        lr=learning_rate,
        eps=1e-8
    )
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * epochs
    )
    loss_fn = CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = input_ids.clone()

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss}")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "HooshvareLab/gpt2-fa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.config.pad_token_id = model.config.eos_token_id


In [ ]:
# Load the GPT-2 model and tokenizer
#model_name = "HooshvareLab/gpt2-fa"
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#model = GPT2LMHeadModel.from_pretrained(model_name)

# Load your dataset using the custom dataset class



vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
poems_file = "Ferdowsi_poem.txt"

In [ ]:
poetry_dataset = FerdowsiPoemDataset(poems_file, tokenizer)
train_dataset, test_dataset = train_test_split(poetry_dataset, test_size=0.1, random_state=42)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: {'input_ids': pad_sequence([item['input_ids'] for item in x], batch_first=True), 'attention_mask': pad_sequence([item['attention_mask'] for item in x], batch_first=True)})
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: {'input_ids': pad_sequence([item['input_ids'] for item in x], batch_first=True), 'attention_mask': pad_sequence([item['attention_mask'] for item in x], batch_first=True)})

# Assuming you have the GPT2 model defined
model = GPT2LMHeadModel.from_pretrained("gpt2")



In [ ]:
num_epochs = 10
train_model(model, train_dataloader, epochs=num_epochs)

Epoch 1: 100%|██████████| 1396/1396 [04:34<00:00,  5.09it/s]


Epoch 1, Average Loss: 4.622444729067193


Epoch 2: 100%|██████████| 1396/1396 [04:35<00:00,  5.07it/s]


Epoch 2, Average Loss: 3.436086690869919


Epoch 3: 100%|██████████| 1396/1396 [04:34<00:00,  5.08it/s]


...

Epoch 8, Average Loss: 1.1307358558164284


Epoch 9: 100%|██████████| 1396/1396 [04:35<00:00,  5.07it/s]


Epoch 9, Average Loss: 0.9417419325144039


Epoch 10: 100%|██████████| 1396/1396 [04:35<00:00,  5.07it/s]

Epoch 10, Average Loss: 0.814685553098818


In [ ]:
fine_tuned_model_path = "fine_tuned.pth"
torch.save(model.state_dict(), fine_tuned_model_path)

In [ ]:
loaded_model = GPT2LMHeadModel.from_pretrained("gpt2")
loaded_model.load_state_dict(torch.load(fine_tuned_model_path))


In [ ]:
def generate_couplet(model, tokenizer, input_text, max_length=60):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=20,
        num_beams=5,
        no_repeat_ngram_size=2,
        top_k=50,
        attention_mask=torch.ones_like(input_ids)
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text


In [ ]:
def preprocess(sample_output):
                gen_sample_output = sample_output.replace("<|startoftext|>", "\n")
                gen_sample_output = gen_sample_output.replace("<s>", "")
                gen_sample_output = gen_sample_output.replace("</s>", "")
                gen_sample_output = gen_sample_output.replace("<sep>", "\n")
                return gen_sample_output


In [ ]:
# Test the model by generating a couplet
input_couplet ="به نام خداوند جان و خرد "

generated_couplet = generate_couplet(model, tokenizer, input_couplet)
print(f"Generated Couplet: {preprocess(generated_couplet)}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Couplet: به نام خداوند جان و خرد له بمرد او از تو برنگذرد زله


In [ ]:
with open("ferdowsi_poem.txt", 'r', encoding='utf-8') as input_file:
    lines = input_file.readlines()
    first=[]
    second=[]
    for i in range(0, 200, 2):
        first.append(lines[i].strip())
        second.append(lines[i+1].strip())



In [ ]:
poemss=["پیش او گوساله بریان آوری","کین جهان چاهیست بس تاریک و تنگ","آسمانی بس بلند و پر ضیا","باقیات الصالحات آمد کریم","این جهان و ساکنانش منتشر","سقف گردون کو چنین دایم بود","ای ضیاء الحق حسام الدین بیار"]

In [ ]:
for i in range(len(poemss)):
  print(preprocess(generate_couplet(model, tokenizer, poemss[i])))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


پیش او گوساله بریان آوری گل و سنبل و خون همچو گشته نراان آوری
کین جهان چاهیست بس تاریک و تنگ بسته به پیش تو آرم به جنگ ون جماعت


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


آسمانی بس بلند و پر ضیاش با دانش و داد و فرش؟ش؟


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


باقیات الصالحات آمد کریمان پر از رنج و دل پر ز حور؟


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


این جهان و ساکنانش منتشرده بود بمردی به کژی و نابخردیستن
سقف گردون کو چنین دایم بود نه آیین او بر چه آیین بودمندیم
ای ضیاء الحق حسام الدین بیاراستند و می و جام و قند Xند


In [ ]:
for i in range(10):
  print(first[i]+" "+second[i])
  print(preprocess(generate_couplet(model, tokenizer, first[i])))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


به نام خداوند جان و خرد کزین برتر اندیشه برنگذرد و ست؟
خداوند نام و خداوند جای خداوند روزی ده رهنمای


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

به بینندگان آفریننده را پیش خواند وزان مهتران با او براندندستن X
نیابد بدو نیز اندیشه راه که او برتر از نام و از جایگاه
نیابد بدو نیز اندیشه راه که او برتر از کوه و هامون و راه؟ست
سخن هر چه زین گوهران بگذرد نیابد بدو راه جان و خرد


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


سخن هر چه زین گوهران بگذرد نیابد بدو راه جان و خردرد؟ستن
خرد گر سخن برگزیند همی همان را گزیند که بیند همی


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


خرد گر سخن برگزیند همی دل از نیک نامی گزیت را سزاست؟ستن
ستودن نداند کس او را چو هست میان بندگی را ببایدت بست
ستودن نداند کس او را چو هست میان بندگی را ببایدت بستافتست
خرد را و جان را همی سنجد اوی در اندیشهٔ سخته کی گنجد اوی
خرد را و جان را همی سنجد اوی در اندیشهٔ سخته کی گنجد اوی X


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# Function to calculate perplexity
def calculate_perplexity(model, data_loader, criterion, device):
    total_loss = 0.0
    total_words = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Calculating Perplexity"):
            inputs, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)

            outputs = model(inputs, attention_mask=attention_mask)
            logits = outputs.logits

            logits = logits.view(-1, logits.size(-1))
            targets = inputs.view(-1)

            loss = criterion(logits, targets)
            total_loss += loss.item()
            total_words += targets.numel()

    perplexity = torch.exp(torch.tensor(total_loss) / torch.tensor(total_words))
    return perplexity.item()

criterion = nn.CrossEntropyLoss()

perplexity = calculate_perplexity(model, test_dataloader, criterion, device)
print(f'Perplexity: {perplexity}')



Calculating Perplexity: 100%|██████████| 156/156 [00:08<00:00, 18.19it/s]

Perplexity: 1.0248655080795288
